In [1]:
import requests
import pandas as pd
import re
import numpy as np
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import os
from argparse import ArgumentParser
from bs4 import BeautifulSoup
import urllib.request
import language_tool_python
from spellchecker import SpellChecker
import requests
import json

## get the url, extract html

In [2]:
#url = 'https://wefarm.com/'
#url='https://about.wefarm.com/terms-and-conditions'
#url='https://www.menuco.co'
url='https://analyticsindiamag.com/how-to-use-bert-transformer-for-grammar-checking/'

In [3]:
response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
response.status_code

200

In [6]:
test_html = response.text

In [7]:
test_html

'<!doctype html>\n<html lang="en-US">\n<head>\n\t<meta charset="UTF-8">\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t<link rel="profile" href="https://gmpg.org/xfn/11">\n\t<meta name=\'robots\' content=\'index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1\' />\n\n\t<!-- This site is optimized with the Yoast SEO Premium plugin v18.4 (Yoast SEO v18.7) - https://yoast.com/wordpress/plugins/seo/ -->\n\t<title>How To Use BERT Transformer For Grammar Checking?</title>\n\t<link rel="canonical" href="https://analyticsindiamag.com/how-to-use-bert-transformer-for-grammar-checking/" />\n\t<meta property="og:locale" content="en_US" />\n\t<meta property="og:type" content="article" />\n\t<meta property="og:title" content="How To Use BERT Transformer For Grammar Checking?" />\n\t<meta property="og:description" content="With the advent of AI, we are witnessing some of the remarkable things which were once deemed impossible now being completely a

## find subgage urls

In [53]:
whole_list=[]
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "html.parser")
for link in soup.findAll('a'):
    if link not in whole_list:
        whole_list.append(link.get('href'))
    else:
        pass
    #print(link.get('href'))

In [54]:
whole_list

['/moja',
 '/login',
 '/about-wefarm',
 '/community-and-shop',
 '/testimonials',
 '/faqs',
 '/contact-us',
 '/join',
 '/moja/join/no-kenyan-number',
 '/login',
 '/join',
 '/moja/join/no-kenyan-number',
 'https://about.wefarm.com/',
 'https://about.wefarm.com/culture',
 'https://about.wefarm.com/contact',
 'https://about.wefarm.com/cookies',
 'https://about.wefarm.com/terms-and-conditions',
 'https://about.wefarm.com/privacy-policy']

In [58]:
subpages = pd.DataFrame(whole_list, columns = ['link'])

In [59]:
subpages=subpages[subpages['link'].str.contains("wefarm")]

In [60]:
subpages=subpages.link.unique()

In [61]:
subpages

array(['/about-wefarm', 'https://about.wefarm.com/',
       'https://about.wefarm.com/culture',
       'https://about.wefarm.com/contact',
       'https://about.wefarm.com/cookies',
       'https://about.wefarm.com/terms-and-conditions',
       'https://about.wefarm.com/privacy-policy'], dtype=object)

## Get text from html

In [8]:
soup = BeautifulSoup(test_html, 'html.parser')

## Clean the text

In [9]:
text = soup.get_text(separator=' ')
text

'\n \n \n \n \n \n \n \n How To Use BERT Transformer For Grammar Checking? \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n   \n   \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n   \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n   Menu \n \n \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Videos \n Hackathons \n Council \n Best Firm \n Discussion \n Contact us \n   \n \n   Menu \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Vi

In [10]:
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)

In [11]:
cleaned_text2 = re.sub("([-+@#^/|*(){}$~`<>=_])|(\[)|(\])|([0-9])", "", text)
#cleaned_text2 = int(text).replace("\n","", text)
cleaned_text2

'\n \n \n \n \n \n \n \n How To Use BERT Transformer For Grammar Checking? \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n   \n   \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n   \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n   Menu \n \n \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Videos \n Hackathons \n Council \n Best Firm \n Discussion \n Contact us \n   \n \n   Menu \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Vi

In [12]:
cleaned_text3 = re.sub("([-+@#^/|*(){}$~`<>=_])|(\[)|(\])|([0-9])", "", text)
cleaned_text3

'\n \n \n \n \n \n \n \n How To Use BERT Transformer For Grammar Checking? \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n   \n   \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n   \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n   Menu \n \n \n Opinions \n Tech Policy \n Developers Corner \n Tech Events \n Careers \n Research \n People & Technology \n Tech Startups \n Education \n News \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Videos \n Hackathons \n Council \n Best Firm \n Discussion \n Contact us \n   \n \n   Menu \n \n \n Who we are \n Advertise \n Hiring Services \n Events \n Vi

In [13]:
cleaned_text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
cleaned_text

'        How To Use BERT Transformer For Grammar Checking                                                                                                    Opinions  Tech Policy  Developers Corner  Tech Events  Careers  Research  People  Technology  Tech Startups  Education  News        Menu    Opinions  Tech Policy  Developers Corner  Tech Events  Careers  Research  People  Technology  Tech Startups  Education  News                                              Who we are  Advertise  Hiring Services  Events  Videos  Hackathons  Council  Best Firm  Discussion  Contact us        Menu    Who we are  Advertise  Hiring Services  Events  Videos  Hackathons  Council  Best Firm  Discussion  Contact us                   Search        Close                SUBSCRIBE                                     Published on \t\t\t\t\t\t\t\t\t\tAugust  \t\t\t\t\t         In    Developers Corner           How To Use BERT Transformer For Grammar Checking               By  \t\t\t\t\t\t\t\t\t\tRavi Tanwar\t\t\

### alternative ways to clean text

In [9]:
requests.get("https://www.menuco.co")

<Response [200]>

In [85]:
x=[]
for i in soup.find_all("p"):
    i.get_text()
    x.append(i.get_text())
        
    print(i.get_text())

I don’t have a Kenyan number
I don’t have a Kenyan number


In [67]:
# for para in soup.find_all("p"):
    print(para.get_text())

I don’t have a Kenyan number
I don’t have a Kenyan number


In [80]:
text2 = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
text2

' Sign up for free  join your farming community  Wefarm Log in About Wefarm Community and Shop Testimonials FAQs Contact us Farming better with Wefarm Learn techniques from fellow farmers Find quality inputs to buy Earn Wefarm points  save money Join I dont have a Kenyan number Already joined log in Meet your farming community Join groups on Wefarmcom and interact with your fellow farmers to solve problems  or help set up your latest agriventure Order products online Wefarmers save precious time by ordering their inputs online Check prices and availability on Wefarm Shop before travelling to the agrovet Get better value with Wefarm points Earn Wefarm Points on every purchase and save money on your future orders Join Wefarm today Join I dont have a Kenyan number Company page Join the team Contact us Cookie Policy Terms of Use  Privacy'

In [81]:
text3=text

In [157]:
def clean(text2):
    """
    takes string as input and
    removes characters inside (),{},[] and <>
    removes characters like -+@#^/|*(){}$~`
    we not not removing ,.!-:;"' as these characters are present in english language 
    """
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text

In [158]:
clean(text)

UnboundLocalError: local variable 'text' referenced before assignment

## spellchecker

In [15]:
dir(SpellChecker)

['_SpellChecker__edit_distance_alt',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_case_sensitive',
 '_check_if_should_check',
 '_distance',
 '_tokenizer',
 '_word_frequency',
 'candidates',
 'correction',
 'distance',
 'edit_distance_1',
 'edit_distance_2',
 'export',
 'known',
 'languages',
 'split_words',
 'unknown',
 'word_frequency',
 'word_probability',
 'word_usage_frequency']

In [16]:
spell=SpellChecker()

In [17]:
splitted_text = cleaned_text2.split()
#x.lower()
print(splitted_text)

['How', 'To', 'Use', 'BERT', 'Transformer', 'For', 'Grammar', 'Checking?', 'Opinions', 'Tech', 'Policy', 'Developers', 'Corner', 'Tech', 'Events', 'Careers', 'Research', 'People', '&', 'Technology', 'Tech', 'Startups', 'Education', 'News', 'Menu', 'Opinions', 'Tech', 'Policy', 'Developers', 'Corner', 'Tech', 'Events', 'Careers', 'Research', 'People', '&', 'Technology', 'Tech', 'Startups', 'Education', 'News', 'Who', 'we', 'are', 'Advertise', 'Hiring', 'Services', 'Events', 'Videos', 'Hackathons', 'Council', 'Best', 'Firm', 'Discussion', 'Contact', 'us', 'Menu', 'Who', 'we', 'are', 'Advertise', 'Hiring', 'Services', 'Events', 'Videos', 'Hackathons', 'Council', 'Best', 'Firm', 'Discussion', 'Contact', 'us', 'Search', 'Close', 'SUBSCRIBE', 'Published', 'on', 'August', ',', 'In', 'Developers', 'Corner', 'How', 'To', 'Use', 'BERT', 'Transformer', 'For', 'Grammar', 'Checking?', 'By', 'Ravi', 'Tanwar', 'With', 'the', 'advent', 'of', 'AI,', 'we', 'are', 'witnessing', 'some', 'of', 'the', 'rema

task: delete recurring words

In [18]:
for word in splitted_text:
    a=[]
    if word != spell.correction(word):
        a.append(spell.correction(word))
    print(f'{word}:{a}')

How:[]
To:[]
Use:[]
BERT:[]
Transformer:[]
For:[]
Grammar:[]
Checking?:['checking']
Opinions:[]
Tech:[]
Policy:[]
Developers:[]
Corner:[]
Tech:[]
Events:[]
Careers:[]
Research:[]
People:[]
&:[]
Technology:[]
Tech:[]
Startups:[]
Education:[]
News:[]
Menu:[]
Opinions:[]
Tech:[]
Policy:[]
Developers:[]
Corner:[]
Tech:[]
Events:[]
Careers:[]
Research:[]
People:[]
&:[]
Technology:[]
Tech:[]
Startups:[]
Education:[]
News:[]
Who:[]
we:[]
are:[]
Advertise:[]
Hiring:[]
Services:[]
Events:[]
Videos:[]
Hackathons:[]
Council:[]
Best:[]
Firm:[]
Discussion:[]
Contact:[]
us:[]
Menu:[]
Who:[]
we:[]
are:[]
Advertise:[]
Hiring:[]
Services:[]
Events:[]
Videos:[]
Hackathons:[]
Council:[]
Best:[]
Firm:[]
Discussion:[]
Contact:[]
us:[]
Search:[]
Close:[]
SUBSCRIBE:[]
Published:[]
on:[]
August:[]
,:[]
In:[]
Developers:[]
Corner:[]
How:[]
To:[]
Use:[]
BERT:[]
Transformer:[]
For:[]
Grammar:[]
Checking?:['checking']
By:[]
Ravi:[]
Tanwar:['anwar']
With:[]
the:[]
advent:[]
of:[]
AI,:['air']
we:[]
are:[]
witnessin

sentencesource:[]
label:[]
labelnotes:[]
sentence:[]
ks:['is']
NaN:[]
The:[]
son:[]
took:[]
care:[]
of:[]
his:[]
parents.:['parents']
bc:['be']
John:[]
wonders:[]
where:[]
him:[]
to:[]
go.:['got']
bc:['be']
NaN:[]
If:[]
Ron:[]
knows:[]
whether:[]
to:[]
wear:[]
a:[]
tuxedo:[]
and:[]
Cas…:['case']
Training:[]
the:[]
BERT:[]
model:[]
involves:[]
the:[]
following:[]
steps::['steps']
Installing:[]
transformers:[]
library:[]
Loading:[]
data:[]
and:[]
tokenizer:[]
then:[]
tokenizing:[]
the:[]
sentences.:['sentences']
Tokenizer:[]
splits:[]
the:[]
sentences:[]
in:[]
words.:['words']
Getting:[]
inputs:[]
ids,:['ids']
segments:[]
ids:[]
and:[]
attention:[]
masks:[]
from:[]
the:[]
ids:[]
Splitting:[]
the:[]
data:[]
into:[]
train:[]
and:[]
validation:[]
sets:[]
Loading:[]
and:[]
choosing:[]
BERT:[]
model:[]
Specifying:[]
optimizer:[]
and:[]
scheduler:[]
Training:[]
and:[]
evaluating:[]
the:[]
model:[]
and:[]
then:[]
getting:[]
inference:[]
based:[]
on:[]
our:[]
examples.:['examples']
For:[]
exampl

betterinformed:[]
and:[]
more:[]
conscious:[]
decisions:[]
about:[]
technology:[]
through:[]
authoritative,:['authoritative']
influential,:['influential']
and:[]
trustworthy:[]
journalism.:['journalism']
Shape:[]
The:[]
Future:[]
of:[]
AI:[]
CONTACT:[]
US:[]
⟶:['i']
Facebook:[]
Twitter:[]
Youtube:[]
Linkedin:['linkedln']
Instagram:[]
About:[]
Us:[]
Advertise:[]
Weekly:[]
Newsletter:[]
Write:[]
for:[]
us:[]
Careers:[]
Contact:[]
Us:[]
Menu:[]
About:[]
Us:[]
Advertise:[]
Weekly:[]
Newsletter:[]
Write:[]
for:[]
us:[]
Careers:[]
Contact:[]
Us:[]
rankings:[]
&:[]
lists:[]
Academic:[]
Rankings:[]
Best:[]
Firms:[]
To:[]
Work:[]
For:[]
Top:[]
Leaders:[]
Emerging:[]
Startups:[]
Trends:[]
PeMa:[]
Quadrant:[]
Menu:[]
Academic:[]
Rankings:[]
Best:[]
Firms:[]
To:[]
Work:[]
For:[]
Top:[]
Leaders:[]
Emerging:[]
Startups:[]
Trends:[]
PeMa:[]
Quadrant:[]
Resources:[]
Python:[]
Libraries:[]
for:[]
data:[]
science:[]
Best:[]
Firms:[]
for:[]
Data:[]
Scientists:[]
Certification:[]
Menu:[]
Python:[]
Librari

## AutoTokenizer

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("addy88/t5-grammar-correction")

model = AutoModelForSeq2SeqLM.from_pretrained("addy88/t5-grammar-correction")

input_ids = tokenizer('grammar: This sentences has has bads grammar.', return_tensors='pt').input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

This sentence has bad grammar.


In [3]:
pip install gingerit

Note: you may need to restart the kernel to use updated packages.


In [5]:
import gingerit

In [4]:
from gingerit.gingerit import GingerIt

text = 'The smelt of fliwers bring back memories.'

parser = GingerIt()
parser.parse(text)

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<html lang="en-US">
   <head>
      <title>Access denied</title>
      <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
      <meta name="robots" content="noindex, nofollow" />
      <meta name="viewport" content="width=device-width,initial-scale=1" />
      <link rel="stylesheet" href="/cdn-cgi/styles/errors.new.min.css" media="screen" />
      <script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedback clicked",properties:{errorCode:1020,helpful:a,version:3}};b.open("POST","https://sparrow.cloudflare.com/api/v1/event");b.setRequestHeader("Content-Type","application/json");b.setRequestHeader("Sparrow-Source-Key","c771f0e4b54944bebf4261d44bd79a1e");
b.send(JSON.stringify(a));c.classList.add("feedback-hidden");d.classList.remo

In [6]:
parser = GingerIt()

text = "I can also help take care of younger kids at playground. At playground, I can help my friend take care of the ball. I also like to help out and think about the feelings of others."

print(parser.parse(text)['result'])

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<html lang="en-US">
   <head>
      <title>Access denied</title>
      <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
      <meta name="robots" content="noindex, nofollow" />
      <meta name="viewport" content="width=device-width,initial-scale=1" />
      <link rel="stylesheet" href="/cdn-cgi/styles/errors.new.min.css" media="screen" />
      <script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedback clicked",properties:{errorCode:1020,helpful:a,version:3}};b.open("POST","https://sparrow.cloudflare.com/api/v1/event");b.setRequestHeader("Content-Type","application/json");b.setRequestHeader("Sparrow-Source-Key","c771f0e4b54944bebf4261d44bd79a1e");
b.send(JSON.stringify(a));c.classList.add("feedback-hidden");d.classList.remo

## language tool python

In [19]:
import language_tool_python

In [27]:
text10 = 'Let’s checck how the two heavyweights of transfor learning perform on checking grammar. Photo by Green Chameleon on Unsplash Transfer Learning came as a game changer for all the NLP researchers, so before diving in let’s quickly revisit what transfer learning is. Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task. Simply it leverages prior knowledge from one domain and task into a different domain and task. '

In [34]:
tool = language_tool_python.LanguageTool('en-US')

tool.correct(text10)

'Let’s check how the two heavyweights of transfer learning perform on checking grammar. Photo by Green Chameleon on Unsplash Transfer Learning came as a game changer for all the NLP researchers, so before diving in let’s quickly revisit what transfer learning is. Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task. Simply it leverages prior knowledge from one domain and task into a different domain and task. '

## split to text into sentences

In [29]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [32]:
splitted_sentences=split_into_sentences(text10)
splitted_sentences

['Let’s checck how the two heavyweights of transfor learning perform on checking grammar.',
 'Photo by Green Chameleon on Unsplash Transfer Learning came as a game changer for all the NLP researchers, so before diving in let’s quickly revisit what transfer learning is.',
 'Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task.',
 'Simply it leverages prior knowledge from one domain and task into a different domain and task.']